In [ ]:
import traci
import sumolib

SUMO_BINARY = "sumo-gui"  # Use "sumo" for non-GUI
SUMO_CONFIG = "busStop.sumocfg"
NET_FILE = "osm_modified.net.xml"  # Replace with actual network file

# Load SUMO network
net = sumolib.net.readNet(NET_FILE)

# Start SUMO
traci.start([SUMO_BINARY, "-c", SUMO_CONFIG,"--start"])

# Get all bus stops
bus_stops = traci.busstop.getIDList()
print(f"Total Bus Stops: {len(bus_stops)}")

bus_stop_edges = {}

for stop in bus_stops:
    lane_id = traci.busstop.getLaneID(stop)
    lane = net.getLane(lane_id)

    # Find all edges connected to this lane
    candidate_edges = [e for e in net.getEdges() if lane in e.getLanes()]

    # Pick the edge that allows buses
    valid_edge = next((e for e in candidate_edges if e.allows("bus")), None)

    if valid_edge:
        bus_stop_edges[stop] = valid_edge
    else:
        print(f"⚠ No valid bus edge found for bus stop {stop}")

print("Bus stops and their edges:", bus_stop_edges)




In [1]:
import traci

SUMO_CONFIG = "busStop.sumocfg"

traci.start(["sumo-gui", "-c", SUMO_CONFIG, "--start"])

bus_stops = traci.busstop.getIDList()

for stop in bus_stops:
    lane_id = traci.busstop.getLaneID(stop)
    edge_id = traci.lane.getEdgeID(lane_id)
    allowed_vehicles = traci.lane.getAllowed(lane_id)

    print(f"🚏 Bus Stop: {stop} | Edge: {edge_id} | Allowed Vehicles: {allowed_vehicles}")

traci.close()


🚏 Bus Stop: -2207611901 | Edge: -205080225#0 | Allowed Vehicles: ('bus', 'tram')
🚏 Bus Stop: 11975930340 | Edge: 158013562#27 | Allowed Vehicles: ('bus',)
🚏 Bus Stop: 11982377441 | Edge: 158014642#3 | Allowed Vehicles: ('bus',)
🚏 Bus Stop: 11982377442 | Edge: 1019992644#1 | Allowed Vehicles: ('bus',)
🚏 Bus Stop: 1702530312 | Edge: 1216156367#2 | Allowed Vehicles: ('bus',)
🚏 Bus Stop: 2141425962 | Edge: 204140137 | Allowed Vehicles: ('bus', 'rail', 'rail_electric', 'rail_fast')
🚏 Bus Stop: 2141425973 | Edge: 204128934#0 | Allowed Vehicles: ('bus', 'rail', 'rail_electric', 'rail_fast')
🚏 Bus Stop: 2207611901 | Edge: 205080226#7 | Allowed Vehicles: ('bus', 'tram')
🚏 Bus Stop: 5189265436 | Edge: 1146853686 | Allowed Vehicles: ('bus', 'subway')
🚏 Bus Stop: 5189265479 | Edge: 535333527#1 | Allowed Vehicles: ('bus', 'subway')
🚏 Bus Stop: 5777572361 | Edge: 610018750#0 | Allowed Vehicles: ('bus', 'subway')
🚏 Bus Stop: 5777572412 | Edge: 1146853675 | Allowed Vehicles: ('bus', 'subway')
🚏 Bus St

KeyboardInterrupt: 

In [ ]:
import xml.etree.ElementTree as ET

def save_routes_to_xml(routes, filename="valid_routes.rou.xml"):
    root = ET.Element("routes")

    for i, route in enumerate(routes):
        route_id = f"route_{i}"
        edges = " ".join(bus_stop_edges[stop].getID() for stop in route)

        # Add route
        route_elem = ET.SubElement(root, "route", id=route_id, edges=edges)

        # Add vehicle following this route
        ET.SubElement(root, "vehicle", id=f"bus_{i}", type="bus", route=route_id, depart="0")

    # Write to XML
    tree = ET.ElementTree(root)
    tree.write(filename)
    print(f"Routes saved to {filename}")

# Generate and save all routes
valid_routes = get_all_valid_routes()
save_routes_to_xml(valid_routes)



In [ ]:
import traci
import xml.etree.ElementTree as ET

# Paths to SUMO files
SUMO_CONFIG = "busStop.sumocfg"   # Change to your SUMO config file
NET_FILE = "osm.net.xml"      # SUMO network file

# Start SUMO in non-GUI mode
traci.start(["sumo", "-c", SUMO_CONFIG, "--start"])

# Get all bus stops dynamically from SUMO
bus_stop_lanes = set()
for stop in traci.busstop.getIDList():
    lane_id = traci.busstop.getLaneID(stop)
    bus_stop_lanes.add(lane_id)

traci.close()  # Close SUMO after fetching bus stops

# Load XML network file
tree = ET.parse(NET_FILE)
root = tree.getroot()

# Modify lanes to allow buses if they contain a bus stop
for edge in root.findall("edge"):
    for lane in edge.findall("lane"):
        lane_id = lane.get("id")
        
        # Check if this lane is used by a bus stop
        if lane_id in bus_stop_lanes:
            allowed = lane.get("allow", "")
            
            # Add "bus" to allowed types if not present
            if "bus" not in allowed:
                new_allowed = f"{allowed} bus".strip()
                lane.set("allow", new_allowed)
                print(f"✅ Bus access added to lane {lane_id} on edge {edge.get('id')}")

# Save the modified network
MODIFIED_NET_FILE = "osm_modified.net.xml"
tree.write(MODIFIED_NET_FILE)
print(f"🚍 Updated network saved as {MODIFIED_NET_FILE}")
